

**Description** 

This Jupyter Notebook contains code for feature engineering and then setting up different model experiments. 

**Author**: Phoebe Ly, Vivek Sakhrani, Isabella Smythe

**Last Updated:** 08-MAR-2025



In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
from rasterio.features import geometry_window
from rasterio.sample import sample_gen
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor



In [ ]:
# Create buffer around HH points
peoplesun_hhapps_anon = pd.read_csv('gs://ads-lacuna-ai4ea/data/dataverse/peoplesun_hhapps_anon.csv') #Make sure data source
eaidgeokey_hh = gpd.read_file('egs://ads-lacuna-ai4ea/data/aidgeokey_hh.geojson')
eaidgeokey_hh['point'] = eaidgeokey_hh['geometry'] 
eaidgeokey_hh['geometry'] = eaidgeokey_hh.to_crs(3857)['geometry'].buffer(2000) #change buffer size if necessary

#Subset df to important variables
df_sub = eaidgeokey_hh[['hhid', 'zone', 'state', 'lga', 'settlement', 'point', '_geo_latitude', '_geo_longitude', 'geometry']]
df_sub = df_sub.to_crs(4326)

### Feature engineering

In [ ]:
#Zonal statistics function

def zonal_calc(raster_file_path, touched, agg, col_name, df):
    with rasterio.open(raster_file_path) as src:
        affine = src.transform
        raster_data = src.read(1)

    with rasterio.open(raster_file_path) as src:
        # Prepare to store zonal stats for each geometry
        agg_values = []
        
        # Process each geometry individually to avoid memory issues
        for geom in df_sub.geometry:
            # Calculate a window around the geometry to avoid loading the full raster
            window = geometry_window(src, [geom])
            
            # Read only the data in the window
            raster_data = src.read(1, window=window)
            
            # Calculate zonal stats for the geometry on the windowed raster data
            stats = zonal_stats(
                [geom],
                raster_data,
                affine=src.window_transform(window),
                stats=[agg],
                nodata=-99999,
                all_touched=touched
            )
            
            # Append the sum result for this geometry
            agg_values.append(stats[0][agg])

        df[col_name] = agg_values
    return df


In [ ]:
#Population

raster_file_path = 'gs://ads-lacuna-ai4ea/data/rasters/population_buffer_clip.tif'
touched = True #specify pixel condition
agg = "sum"
col_name = 'tot_population'
df_sub = zonal_calc(raster_file_path, touched, agg, col_name, df_sub)

In [ ]:
# Temperature

raster_file_path = 'gs://ads-lacuna-ai4ea/data/rasters/temperature_merge.tif'
touched = True #specify pixel condition
agg = "median"
col_name = 'med_temperature'
df_sub = zonal_calc(raster_file_path, touched, agg, col_name, df_sub)

In [ ]:
# Precipitation

raster_file_path = 'gs://ads-lacuna-ai4ea/data/rasters/precipitation_merge.tif'
touched = True #specify pixel condition
agg = "median"
col_name = 'med_precipitation'
df_sub = zonal_calc(raster_file_path, touched, agg, col_name, df_sub)


In [ ]:
# AWI

raster_file_path = 'gs://ads-lacuna-ai4ea/data/rasters/awi_africa4326.tif'
touched = True #specify pixel condition
agg = "median"
col_name = 'med_awi'
df_sub = zonal_calc(raster_file_path, touched, agg, col_name, df_sub)

In [ ]:
# Spending

raster_file_path = 'gs://ads-lacuna-ai4ea/data/rasters/spending_nga4326.tif'
touched = True #specify pixel condition
agg = "median"
col_name = 'med_spending'
df_sub = zonal_calc(raster_file_path, touched, agg, col_name, df_sub)

In [ ]:
#URCA #sampling pixel

raster_file_path = 'gs://ads-lacuna-ai4ea/data/rasters/Urban-Rural Catchment Areas (URCA).tif'
coordinates = [(point.x, point.y) for point in df_sub.point]
with rasterio.open(raster_file_path) as src:
    sampled_values = list(src.sample(coordinates))

df_sub['urca'] = [val[0] for val in sampled_values]

df_sub = pd.get_dummies(df_sub, columns=['urca'], prefix='urca') # one hot encode

In [ ]:
#For electrification layer

# Define a custom function to count occurrences of value 2
def count_value_2(values):
    return np.sum(values == 2)

# Define a custom function to count total valid pixels (excluding NoData)
def count_total_valid(values):
    return np.sum(~np.isnan(values))


In [ ]:
#Electrification

def elec_calc(raster_path, vector_gdf):
    print(raster_path) #show that it's looping

    with rasterio.open(raster_path) as src:
        raster_crs = src.crs
        raster_transform = src.transform
        raster_nodata = src.nodata
        raster_dtype = src.dtypes[0]
        raster_shape = (src.height, src.width)
        raster_bounds = src.bounds
        
        if vector_gdf.crs != raster_crs:
            vector_gdf = vector_gdf.to_crs(raster_crs)
            print("Reprojected vector data to match raster CRS.")

        stats = zonal_stats(
            vector_gdf,
            raster_path,
            categorical=True,
            nodata=raster_nodata,
            geojson_out=False
        )

        proportions = []
        for stat in stats:
            count_2 = stat.get(2, 0)  # Get count of value '2'; default to 0 if not present
            total_pixels = sum(stat.values())
            proportion = count_2 / total_pixels if total_pixels > 0 else np.nan
            proportions.append(proportion)

        # Add the proportions to the GeoDataFrame
        col_name = 'electrification_prop_' + raster_path[46:48]
        vector_gdf[col_name] = proportions

        return vector_gdf


In [ ]:
# Define file paths and calculate electrification proportion

date_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for date in date_list:
    raster_path = 'gs://ads-lacuna-ai4ea/data/rasters/Electrification-1000m_0_9_africa_2021-'+date+'.tif' 
    df_sub = elec_calc(raster_path, df_sub)


In [ ]:
# Checking if electrification status has changed within households (if it has not, only use one month)
unique_counts = df_sub.iloc[:, -12:].nunique(axis=1)
sum(unique_counts ==1) # Amount of HH whos electricity status has not changed